# Predicting energies efficiently and on mixed-sites

In [ ]:
import xgboost as xgb
import numpy as np
import pandas as pd

In [ ]:
#Load H binding energy prediction model
H_DFT_model = xgb.Booster({'nthread': 8})
H_DFT_model.load_model("../models/"+"H_DFT.model")

#Load COOH binding energy prediction model
COOH_DFT_model = xgb.Booster({'nthread': 8})
COOH_DFT_model.load_model("../models/"+"COOH_DFT.model")

models = {"H": H_DFT_model, "COOH": COOH_DFT_model}

### Compare time to make predictions of COOH and H on a 100 by 100 surface with the new way to predict energies

#### Old method:

#### New method: